# Fuzzy Cognitive Maps - Scenario

Fuzzy Cognitive Maps (FCM) is a method, that allows to use domain knowledge of experts and explore different outcomes or behaviour based on hypothetical scenarios. With the gained knowledge of running different scenarios, potential pitfalls or unwanted effects can be detected, before actions are deployed to the real world.

For this notebook the python package `fcmpy` is used.

## Simulation
Let us consider a simple example to illustrate the simulation option with FCMs.

* Graph nodes denote concepts or variables that describe the problem
* Graph edges represent (casual) relationships between concepts, which are characterized by positive or negative weights
* Weights can take values in the `[-1, 1]` interval so that they represent both the direction and the intensity of causal relationshipts

We define the causal relationships manually according to the rules that govern the simplified food chain. For example, predators thrive in the area when ther is a lot of prey. And grass, which serves as food source for predator, has a positive effect on them.

![FCM Visualization](image/FCM_Vis.png)

Here we load the necessary packages and functions

In [2]:
from fcmpy import FcmSimulator
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'fcmpy'

The shown edges in the images, are stored as a `weight_matrix`.

In [ ]:
weight_matrix = pd.DataFrame(np.array([[0, 1, -1],[-1,0,0], [1,0,0]]),
                             columns=['Prey', 'Predator', 'Grass'])
weight_matrix

In our example we want simulate what happens **if**:
* we have **medium** number of predators, which will be represented as **0.5**
* we have a **moderate** number of grass, which will be represented as **0.75**

In [ ]:
init_state = {'Predator': 0.5,'Prey': 0.5,"Grass": 0.75}

Initalise the simulation function `FcmSimulator` from `fcmpy`.

In [ ]:
sim = FcmSimulator()

### Different Activation functions

**Sigmoid** <br>
In our first scenario the simulation is done with the `sigmoid` activation function. The values converge after 9 iterations, which indicates a stable pattern (equilibrium).

**TanH** <br>
In the second scenario the simulation is done with the `tanh` activation function. Here cyclic behaviour can be seen, which does not converge. In our example it is a repeating pattern.

In [ ]:
res_sigmoid = sim.simulate(initial_state=init_state, weight_matrix=weight_matrix, transfer='sigmoid', inference='mKosko', thresh=0.001, iterations=50, l=1)

In [ ]:
res_hyperbolic = sim.simulate(initial_state=init_state, weight_matrix=weight_matrix, transfer='tanh',
                           inference='mKosko', thresh=0.001, iterations=50, l=1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two plots side-by-side

res_sigmoid.plot(ax=axes[0], marker='o')
axes[0].get_legend().remove()
axes[0].set_xlabel("Iteration Number")
axes[0].set_title("Sigmoid function")  # Individual title for left plot

res_hyperbolic.plot(ax=axes[1], marker='o')
axes[1].legend(bbox_to_anchor=(1, 0.94))
axes[1].set_xlabel("Iteration Number")
axes[1].set_title("Tanh function")  # Individual title for right plot

#fig.text(0.5, 0.02, "Different Components for the Same Models", ha="center", fontsize=12)

# Step 4: Adjust layout for spacing
plt.tight_layout(rect=[0, 0.03, 1, 1])  # Leave space for the shared x-label

# Step 5: Show the plots
plt.show()


## Intervention

After getting a feeling for the simulation of a scenario. Here we take a look at possible interventions

In [ ]:
from fcmpy import FcmIntervention

In [ ]:
inter = FcmIntervention(FcmSimulator)

In [ ]:
inter.initialize(initial_state=init_state, weight_matrix=weight_matrix,transfer="tanh", inference="mKosko",thresh=0.001, iterations=50, l=1)

In [ ]:
inter.test_results["baseline"]

Here we add an interventen, where hunters are allowed to remove some predators per season. This is why kind of intervention is remarked `continuous`. 

In [ ]:
inter.add_intervention("Intervention_Hunters", type="continuous", impact={"Predator":-0.3}, effectiveness=1)

In [ ]:
inter.test_intervention("Intervention_Hunters", iterations=50)

In [ ]:
intervention_hunter = inter.test_results["Intervention_Hunters"][["Prey", "Predator", "Grass"]]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two plots side-by-side

res_hyperbolic.plot(ax=axes[0], marker='o')
axes[0].get_legend().remove()
axes[0].set_xlabel("Iteration Number")
axes[0].set_title("Without Intervention")  # Individual title for left plot

intervention_hunter.plot(ax=axes[1], marker='o')
axes[1].legend(bbox_to_anchor=(1, 0.94))
axes[1].set_xlabel("Iteration Number")
axes[1].set_title("With Intervention")  # Individual title for right plot

#fig.text(0.5, 0.02, "Different Components for the Same Models", ha="center", fontsize=12)

# Step 4: Adjust layout for spacing
plt.tight_layout(rect=[0, 0.03, 1, 1])  # Leave space for the shared x-label

# Step 5: Show the plots
plt.show()
